In [ ]:
# For the check that indeed the pipeline works!
import os
# ────────────────────────────────────────────────
import env_config
# ────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt

# your style module
import thesis_plot_style as tps

# your sim-binning pipeline
from binner_sims import binner_sims
import parfiles.noNoise.parfile as par

# ────────────────────────────────────────────────
# Settings
bin_type = "agr2"
nsims    = 240
# ────────────────────────────────────────────────

def get_noiseless_sims(par, bin_type):
    b = binner_sims('p', 'p', par, bin_type)
    sims = b.get_sims_bandpowers() - b.get_mcn0() - b.get_n1()
    return b.bin_lavs, sims

# compute fiducial + sims
b0        = binner_sims('p', 'p', par, bin_type)
ell, fid  = b0.bin_lavs, b0.fid_bandpowers
ell_sim, sims = get_noiseless_sims(par, bin_type)

# ────────────────────────────────────────────────
with tps.context_two_panel(filename="fig_dual.png", heights=(2/3,1/3), hspace=0) as (fig, (ax1, ax2)):    
    # immediately zero‐out the gutter

    # top: power spectrum
    ax1.plot(ell, fid, 'k--', lw=1.6, label=r"Fiducial $C_\ell^{\phi\phi}$")
    ax1.plot(ell_sim, sims,
             color=tps.CB_PALETTE[0], lw=1.6,
             label=f"{nsims} noiseless sims")
    ax1.set_ylabel(r"$C_\ell^{\phi\phi}$")
    ax1.legend(loc="upper right")
    ax1.set_title("Lensing Power Spectrum & Residuals")

    # bottom: fractional difference
    frac = (sims - fid) / fid
    ax2.plot(ell, frac, color=tps.CB_PALETTE[1], lw=1.6)
    ax2.axhline(0, color="gray", lw=0.8, ls="--")
    # Draw shaded ±1% band
    ax2.fill_between(
        [0, 1200],
        0.01, -0.01,
        color="gray",
        alpha=0.2,
        zorder=0,
        label=r"±1\%"
    )
    ax2.set_xlabel(r"$\ell$")
    ax2.set_ylabel(r"$\Delta C_\ell / C_\ell^{\phi\phi}$")
    ax2.set_xlim([0,1200])


In [ ]:
#!/usr/bin/env python3
import os
# ─── environment setup ─────────────────────────────────────────────
import env_config
# ────────────────────────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt
from binner_sims import binner_sims
import thesis_plot_style as tps

# parameter files
import parfiles.noNoise.parfile            as par_nn
import parfiles.noNoise.parfile_delensed   as par_nn_del
import parfiles.noNoise.parfile_delensed_qest    as par_nn_qest
import parfiles.noNoise.parfile_delensed_PolQEST as par_nn_polq

import parfiles.Noise.parfile            as par_n
import parfiles.Noise.parfile_delensed   as par_n_del
import parfiles.Noise.parfile_delensed_qest    as par_n_qest
import parfiles.Noise.parfile_delensed_PolQEST as par_n_polq

# common settings
bin_type = "fullA_20"
scenarios = [
    (par_nn,    "MV: lensed",                  'p'),
    (par_nn_del,r"MV: delensed with input $\kappa_{LM}$", 'p'),
    (par_nn_qest, "MV: internally delensed with MV-QEST", 'p'),
    (par_nn_polq, "TT: internally delensed with Pol-QEST", 'ptt'),
]
noisy_scenarios = [
    (par_n,      "MV: lensed",                  'p'),
    (par_n_del,  r"MV: delensed with input $\kappa_{LM}$", 'p'),
    (par_n_qest,"MV: internally delensed with MV-QEST", 'p'),
    (par_n_polq,"TT: internally delensed with Pol-QEST", 'ptt'),
]

# apply thesis style
tps.apply_style("single")

# precompute bin boundaries & centers for visual guides
bins_l = np.array([8, 21, 40, 66, 101, 145, 199, 264, 339, 426, 526, 638, 763, 902])
bins_u = np.array([20, 39, 65, 100, 144, 198, 263, 338, 425, 525, 637, 762, 901, 2048])
bin_centers   = 0.5 * (bins_l + bins_u)
bin_halfwidth = 0.5 * (bins_u - bins_l)

# ─── Noiseless figure ────────────────────────────────────────
with tps.context_figure(filename="fig_clpp_noiseless.png") as (fig, ax):
    # fiducial (no error bars)
    b0 = binner_sims('p','p', par_nn, bin_type)
    ell = b0.bin_lavs
    ax.plot(ell, b0.fid_bandpowers,
            'k--', label="fiducial", lw=1.6)

    # lightly shade the bins
    # for l,u in zip(bins_l, bins_u):
    #     ax.axvspan(l, u, color="gray", alpha=0.05, zorder=0)

    # Monte Carlo scenarios with error bars
    for ci, (pf, label, kPhi) in enumerate(scenarios):
        b = binner_sims(kPhi, kPhi, pf, bin_type)

        # get mean and 1σ-on-the-mean
        bp_mean, bp_err = b.get_sims_bandpowers_with_error()
        # subtract biases
        bp_mean -= (b.get_mcn0() + b.get_n1())

        # plot with errorbars
        color = tps.CB_PALETTE[ci+2]
        ax.errorbar(ell, bp_mean,
                    yerr=bp_err,
                    fmt='o-', lw=1.6, capsize=3,
                    markersize=4.5,
                    color=color,
                    label=label)

    ax.axhline(0, color="gray", lw=0.8, ls="--")
    ax.set(xlabel=r"$\ell$",
           ylabel=r"$C_\ell^{\phi\phi}$",
           title="Noiseless $C_\ell^{\phi\phi}$ Estimates")
    # ax.set_xlim([0, ell[-1]])
    ax.set_xlim([0,400])
    ax.legend(loc="upper right")

# ─── Noisy figure ────────────────────────────────────────────
with tps.context_figure(filename="fig_clpp_noisy.png") as (fig, ax):
    # fiducial + calibration
    b0 = binner_sims('p','p', par_n, bin_type)
    ell = b0.bin_lavs
    fid = b0.fid_bandpowers
    sims_raw = b0.get_sims_bandpowers() - b0.get_mcn0() - b0.get_n1()
    calib = fid / sims_raw  # vector of per-bin calibration

    ax.plot(ell, fid, 'k--', label="fiducial")
    # for l,u in zip(bins_l, bins_u):
    #     ax.axvspan(l, u, color="gray", alpha=0.05, zorder=0)

    # Monte Carlo scenarios with error bars, scaled by calib
    for ci, (pf, label, kPhi) in enumerate(noisy_scenarios):
        b = binner_sims(kPhi, kPhi, pf, bin_type)

        bp_mean, bp_err = b.get_sims_bandpowers_with_error()
        bp_mean -= (b.get_mcn0() + b.get_n1())

        # apply the same calibration factor to mean AND error
        bp_mean *= calib
        bp_err  *= calib

        color = tps.CB_PALETTE[ci+2]
        ax.errorbar(ell, bp_mean,
                    yerr=bp_err,
                    fmt='o-', lw=1.6, capsize=3,
                    markersize=4.5,
                    color=color,
                    label=label)

    ax.axhline(0, color="gray", lw=0.8, ls="--")
    ax.set(xlabel=r"$\ell$",
           ylabel=r"$C_\ell^{\phi\phi}$",
           title="Noisy $C_\ell^{\phi\phi}$ Estimates")
    # ax.set_xlim([0, ell[-1]])
    ax.set_xlim([0,400])
    ax.legend(loc="upper right")


Using lenspyx alm2map
Binner sims, calculating SIGMA...


100%|██████████| 240/240 [00:00<00:00, 284.60it/s]


Binner sims, calculating SIGMA...


100%|██████████| 240/240 [00:02<00:00, 89.51it/s]


Binner sims, calculating SIGMA...


...

100%|██████████| 240/240 [00:02<00:00, 85.66it/s]


Binner sims, calculating SIGMA...


100%|██████████| 240/240 [00:02<00:00, 83.36it/s]


Binner sims, calculating SIGMA...


100%|██████████| 240/240 [00:02<00:00, 80.11it/s]


In [13]:
#!/usr/bin/env python3
import os
# ─── environment setup ─────────────────────────────────────────────
os.environ['PLENS'] = "/mnt/d/THESIS/PLENS"
os.environ['INPUT'] = "/mnt/d/THESIS/SIMS"
os.chdir("/home/gilles/thesis/notebooks")
# ────────────────────────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt
from binner_sims import binner_sims
import thesis_plot_style as tps

# Import all parameter files to compare
import parfiles.noNoise.parfile              as par_nn_lensed
import parfiles.noNoise.parfile_delensed     as par_nn_del
import parfiles.noNoise.parfile_delensed_qest as par_nn_qest
import parfiles.noNoise.parfile_delensed_PolQEST as par_nn_polq

cmp = tps.CB_PALETTE

# Define scenarios: (parfile, estimator_key, label, style)
scenarios = [
    (par_nn_lensed, 'p',    "Lensed",                           {'linestyle':'-',  'color':cmp[2]}),
    (par_nn_del,     'p',    r"Delensed $\kappa_{LM}$", {'linestyle':'--','color':cmp[3]}),
    (par_nn_qest,    'p',    "Delensed MV-QE", {'linestyle':'-.','color':cmp[4]}),
    (par_nn_polq,    'ptt',  "Delensed Pol-QE", {'linestyle':':', 'color':cmp[5]}),
]

# Use agr2 grid for unbinned ℓ-values
grid_btype = "agr2"

# Create two-panel figure
tps.apply_style("single")
with tps.context_two_panel(filename="fig_phi_all_n0_unbinned.png", heights=(2/3,2/3), hspace=0.02) as (fig, (ax1, ax2)):
    # Upper: uncorrected phi estimates
    for parfile, key, label, style in scenarios:
        b = binner_sims(key, key, parfile, btype=grid_btype)
        ell    = b.bin_lavs
        phi_uc = b.get_sims_bandpowers()   # uncorrected <ϕϕ>
        ax1.plot(ell, phi_uc,
                 label=label,
                 lw=1.6,
                 **style)

    ax1.set_ylabel(r"$C_\ell^{\hat\phi\hat\phi}$")
    ax1.set_xlim([ell.min(), ell.max()])
    ax1.axhline(0, color="gray", lw=0.8, ls="--")
    ax1.set_title("Uncorrected QE Output (top) & N0 bias (lower)")
    ax1.legend(loc="upper left", fontsize=10)

    # Lower: N0 biases
    for parfile, key, label, style in scenarios:
        b = binner_sims(key, key, parfile, btype=grid_btype)
        ell = b.bin_lavs
        n0  = b.get_mcn0()               # N0 bias
        ax2.plot(ell, n0,
                 label=label,
                 lw=1.6,
                 **style)

    ax2.set_xlabel(r"$\ell$")
    ax2.set_ylabel(r"$N_\ell^{(0)}$")
    ax2.set_xlim([ell.min(), ell.max()])
    ax2.axhline(0, color="gray", lw=0.8, ls="--")
    # ax2.set_title("N0 Bias")
    # ax2.legend(loc="upper right")

# Display the figure
# plt.show()



/home/gilles/thesis/notebooks/thesis_plot_style.py:94: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


In [ ]:
# Plotting all the cl_pp spectra
import os
# ─── your existing environment setup ────────────────────────
import env_config
# ─────────────────────────────────────────────────────────────

import numpy as np
from binner_sims import binner_sims
import thesis_plot_style as tps

# your parameter files
import parfiles.noNoise.parfile            as par_nn
import parfiles.noNoise.parfile_delensed   as par_nn_del
import parfiles.noNoise.parfile_delensed_qest    as par_nn_qest
import parfiles.noNoise.parfile_delensed_PolQEST as par_nn_polq

import parfiles.Noise.parfile            as par_n
import parfiles.Noise.parfile_delensed   as par_n_del
import parfiles.Noise.parfile_delensed_qest    as par_n_qest
import parfiles.Noise.parfile_delensed_PolQEST as par_n_polq

# common settings
bin_type = "agr2"
scenarios = [
    (par_nn,    "MV: lensed",                  'p'),
    (par_nn_del,"MV: delensed with input KLM", 'p'),
    (par_nn_qest, "MV: internally delensed with MV-QEST", 'p'),
    (par_nn_polq, "TT: internally delensed with Pol-QEST", 'ptt'),
]
noisy_scenarios = [
    (par_n,      "MV: lensed",                  'p'),
    (par_n_del,  "MV: delensed with input KLM", 'p'),
    (par_n_qest,"MV: internally delensed with MV-QEST", 'p'),
    (par_n_polq,"TT: internally delensed with Pol-QEST", 'ptt'),
]

# apply thesis style
tps.apply_style("single")

# ─── Noiseless figure ────────────────────────────────────────
with tps.context_figure(filename="fig_clpp_noiseless.png") as (fig, ax):
    # fiducial
    b0 = binner_sims('p','p', par_nn, bin_type)
    ell = b0.bin_lavs
    ax.plot(ell, b0.fid_bandpowers, 'k--', label="fiducial", lw=1.6)
    # Inside your figure block, after computing bin edges:
    bins_l = np.array([8, 21, 40, 66, 101, 145, 199, 264, 339, 426, 526, 638, 763, 902])
    bins_u = np.array([20, 39, 65, 100, 144, 198, 263, 338, 425, 525, 637, 762, 901, 2048])
    bin_centers = 0.5 * (bins_l + bins_u)
    bin_halfwidths = 0.5 * (bins_u - bins_l)

    for l, u in zip(bins_l, bins_u):
        ax.axvspan(l, u, color="gray", alpha=0.05, zorder=0)

    # each scenario
    cbp = 0
    for pf, label, kPhi in scenarios:
        b = binner_sims(kPhi, kPhi, pf, bin_type)
        sims_bp = b.get_sims_bandpowers() - b.get_mcn0() - b.get_n1()
        ax.plot(ell, sims_bp, lw=1.6, label=label, color=tps.CB_PALETTE[cbp])
        cbp+=1

    ax.axhline(0, color="gray", lw=0.8, ls="--")
    ax.set(
        xlabel=r"$\ell$",
        ylabel=r"$C_\ell^{\phi\phi}$",
        title="Noiseless $C_\ell^{\phi\phi}$ Estimates",
    )
    ax.legend(loc="upper right")
    ax.set_xlim([0, b0.bin_lavs[-1]])
    # ax.set_yscale("log")

# ─── Noisy figure ────────────────────────────────────────────
with tps.context_figure(filename="fig_clpp_noisy.png") as (fig, ax):
    # fiducial + calibration factor
    b0 = binner_sims('p','p', par_n, bin_type)
    ell = b0.bin_lavs
    fid = b0.fid_bandpowers
    sims_raw = b0.get_sims_bandpowers() - b0.get_mcn0() - b0.get_n1()
    calib = fid / sims_raw

    ax.plot(ell, fid, 'k--', label="fiducial")

    for l, u in zip(bins_l, bins_u):
        ax.axvspan(l, u, color="gray", alpha=0.05, zorder=0)

    # each noisy scenario
    for pf, label, kPhi in noisy_scenarios:
        b = binner_sims(kPhi, kPhi, pf, bin_type)
        sims_bp = (b.get_sims_bandpowers() - b.get_mcn0() - b.get_n1()) * calib
        ax.plot(ell, sims_bp, lw=1.6, label=label)
    ax.axhline(0, color="gray", lw=0.8, ls="--")
    ax.set(
        xlabel=r"$\ell$",
        ylabel=r"$C_\ell^{\phi\phi}$",
        title="Noisy $C_\ell^{\phi\phi}$ Estimates",
    )
    ax.legend(loc="upper right")
    ax.set_xlim([0, b0.bin_lavs[-1]])
    # ax.set_yscale("log")


Using lenspyx alm2map


In [ ]:
#!/usr/bin/env python3
import os
# ─── environment setup ─────────────────────────────────────────────
import env_config
# ────────────────────────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt
from binner_sims import binner_sims
import thesis_plot_style as tps

# parameter‐file aliases
import parfiles.noNoise.parfile                        as par_nn
import parfiles.noNoise.parfile_delensed_PolQEST       as par_nn_polq
import parfiles.Noise.parfile                  as par_n
import parfiles.Noise.parfile_delensed_PolQEST as par_n_polq

# common settings
bin_type = "fullA_10"
ests_wf  = ("p_p", "p_p")    # Pol‐QE for Wiener filter
ests_MV  = ("p",   "p")      # MV‐QE for lensing baseline
ests_TT  = ("ptt", "ptt")    # TT‐QE for delensed spectrum

# compute both cases
cases = {
    "Noiseless": (par_nn,   par_nn_polq),
    "Noisy":     (par_n,    par_n_polq),
}

results = {}
for label, (par_base, par_del) in cases.items():
    # Wiener filter (no error bars)
    bw = binner_sims(*ests_wf, par_base, bin_type)
    ell = bw.bin_lavs
    W   = bw.get_WF()
    

    # ----------------------------------------------------------------
    # 1) MV QE lensing bandpowers + error
    bl = binner_sims(*ests_MV, par_base, bin_type)
    C_len, C_len_err = bl.get_sims_bandpowers_with_error()  

    # subtract biases (no error on biases here)
    C_len -= bl.get_mcn0() + bl.get_n1()

    # 2) TT-QE delensed bandpowers + error
    bd = binner_sims(*ests_TT, par_del, bin_type)
    C_del, C_del_err = bd.get_sims_bandpowers_with_error()

    C_del -= bd.get_mcn0() + bd.get_n1()

    # 3) Efficiency + its propagated 1σ
    eff_mean = 1.0 - (C_del / C_len)
    eff_err  = np.sqrt(
        (C_del_err / C_len)**2 +
        (C_del * C_len_err / C_len**2)**2
    )

    results[label] = (ell, W, eff_mean, eff_err)

# plot them all together
tps.apply_style("single")
with tps.context_figure(filename="fig_wf_vs_eff_combined.png") as (fig, ax):
    ax.axvspan(25, 35, color="gray", alpha=0.2, zorder=0)

    for i, (label, (ell, W, eff, err)) in enumerate(results.items()):
        cmap = tps.CB_PALETTE

        # plot Wiener filter as before
        ax.plot(np.arange(len(W)), W,
                color=cmap[2 + 2*i], lw=1.6,
                label=f"Wiener $W_\\ell$ ({label})")

        # plot efficiency with error bars
        ax.errorbar(ell, eff, yerr=err,
                    fmt='o', ls='--', lw=1.2, capsize=3,
                    color=cmap[3 + 2*i],
                    label=f"Eff.~$1 - C_\\ell^{{\\rm del}}/C_\\ell^{{\\rm len}}$ ({label})")


    ax.hlines(0, 0,300,'grey', '--', alpha=0.3)
    ax.set_xlim(0, 300)
    ax.set_xlabel(r"$\ell$")
    ax.set_ylabel("Dimensionless")
    ax.set_ylim([-.1, 0.6])
    ax.set_title("Pol-QE Wiener Filter vs Delensing Efficiency\n(Noiseless & Noisy)")
    ax.legend(loc="upper right", frameon=False)


Binner sims, calculating SIGMA...


100%|██████████| 240/240 [00:07<00:00, 31.83it/s]


Binner sims, calculating SIGMA...


100%|██████████| 240/240 [00:10<00:00, 22.34it/s]


Binner sims, calculating SIGMA...


100%|██████████| 240/240 [00:09<00:00, 24.34it/s]


Binner sims, calculating SIGMA...


100%|██████████| 240/240 [00:11<00:00, 21.80it/s]
